<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка-данных" data-toc-modified-id="Подготовка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка данных</a></span></li><li><span><a href="#Исследование-задачи" data-toc-modified-id="Исследование-задачи-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Исследование задачи</a></span></li><li><span><a href="#Борьба-с-дисбалансом" data-toc-modified-id="Борьба-с-дисбалансом-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Борьба с дисбалансом</a></span><ul class="toc-item"><li><span><a href="#Class-weight" data-toc-modified-id="Class-weight-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Class weight</a></span></li><li><span><a href="#Upsample" data-toc-modified-id="Upsample-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Upsample</a></span></li><li><span><a href="#Downsample" data-toc-modified-id="Downsample-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Downsample</a></span></li></ul></li><li><span><a href="#Тестирование-модели" data-toc-modified-id="Тестирование-модели-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Тестирование модели</a></span></li><li><span><a href="#Чек-лист-готовности-проекта" data-toc-modified-id="Чек-лист-готовности-проекта-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист готовности проекта</a></span></li></ul></div>

# Отток клиентов

Из «Бета-Банка» стали уходить клиенты. Каждый месяц. Немного, но заметно. Банковские маркетологи посчитали: сохранять текущих клиентов дешевле, чем привлекать новых.

Нужно спрогнозировать, уйдёт клиент из банка в ближайшее время или нет. Нам предоставлены исторические данные о поведении клиентов и расторжении договоров с банком. 

Постройте модель с предельно большим значением *F1*-меры. 

Источник данных: [https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling](https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling)

Признаки
- RowNumber — индекс строки в данных
- CustomerId — уникальный идентификатор клиента
- Surname — фамилия
- CreditScore — кредитный рейтинг
- Geography — страна проживания
- Gender — пол
- Age — возраст
- Tenure — сколько лет человек является клиентом банка
- Balance — баланс на счёте
- NumOfProducts — количество продуктов банка, используемых клиентом
- HasCrCard — наличие кредитной карты
- IsActiveMember — активность клиента
- EstimatedSalary — предполагаемая зарплата

Целевой признак
- Exited — факт ухода клиента

## Подготовка данных

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, accuracy_score, f1_score, recall_score, precision_score, roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier
from sklearn.utils import shuffle
from sklearn.model_selection import GridSearchCV

In [3]:
df = pd.read_csv('/datasets/Churn.csv')
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>

RangeIndex: 10000 entries, 0 to 9999

Data columns (total 14 columns):

 #   Column           Non-Null Count  Dtype  

---  ------           --------------  -----  

 0   RowNumber        10000 non-null  int64  

 1   CustomerId       10000 non-null  int64  

 2   Surname          10000 non-null  object 

 3   CreditScore      10000 non-null  int64  

 4   Geography        10000 non-null  object 

 5   Gender           10000 non-null  object 

 6   Age              10000 non-null  int64  

 7   Tenure           9091 non-null   float64

 8   Balance          10000 non-null  float64

 9   NumOfProducts    10000 non-null  int64  

 10  HasCrCard        10000 non-null  int64  

 11  IsActiveMember   10000 non-null  int64  

 12  EstimatedSalary  10000 non-null  float64

 13  Exited           10000 non-null  int64  

dtypes: float64(3), int64(8), object(3)

memory usage: 1.1+ MB


Как видим, столбцы записаны в CamelCase, а в стольбце `Tenure` присутсвуют пропуски.

In [5]:
df = df.rename(columns={'RowNumber':'row_number', 'CustomerId':'customer_id', 'CreditScore':'credit_score', 
                        'NumOfProducts':'num_of_products', 'HasCrCard':'has_cr_card', 'IsActiveMember':'is_active_member',
                       'EstimatedSalary':'estimated_salary'})
df.columns = df.columns.str.lower()

Привели название столбцов к змениному регистру.

In [6]:
df[df['tenure'].isna()]

,row_number,customer_id,surname,credit_score,geography,gender,age,tenure,balance,num_of_products,has_cr_card,is_active_member,estimated_salary,exited
30,31,15589475,Azikiwe,591,Spain,Female,39,NaN,0.00,3,1,0,140469.38,1
48,49,15766205,Yin,550,Germany,Male,38,NaN,103391.38,1,0,1,90878.13,0
51,52,15768193,Trevisani,585,Germany,Male,36,NaN,146050.97,2,0,0,86424.57,0
53,54,15702298,Parkhill,655,Germany,Male,41,NaN,125561.97,1,0,0,164040.94,1
60,61,15651280,Hunter,742,Germany,Male,35,NaN,136857.00,1,0,0,84509.57,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9944,9945,15703923,Cameron,744,Germany,Male,41,NaN,190409.34,2,1,1,138361.48,0
9956,9957,15707861,Nucci,520,France,Female,46,NaN,85216.61,1,1,0,117369.52,1
9964,9965,15642785,Douglas,479,France,Male,34,NaN,117593.48,2,0,0,113308.29,0
9985,9986,15586914,Nepean,659,France,Male,36,NaN,123841.49,2,1,0,96833.00,0


In [7]:
df['tenure'].value_counts()

1.0     952
2.0     950
8.0     933
3.0     928
5.0     927
7.0     925
4.0     885
9.0     882
6.0     881
10.0    446
0.0     382
Name: tenure, dtype: int64

Данный столбец отражает количество лет, которые человек является клиентом банка. Данные находятся в диапозоне от 0 до 10. Причины появления пропусков неясны. Для построения модели заполним пропуски нулями.

In [8]:
df['tenure'] = df['tenure'].fillna(0)

Пропуски в столбце `tenure` заполнены.

In [14]:
df_ohe = df.join(pd.get_dummies(df[['geography', 'gender']], drop_first=True))
df_ohe = df_ohe.drop(columns = ['geography', 'gender'], axis = 1)

Провели прямое кодирование категориальных переменных, чтобы избежать дамми-ловушку, удалили первые столбцы.

In [9]:
features = df_ohe.drop(columns = ['exited', 'row_number', 'surname', 'customer_id'], axis = 1)
target = df_ohe['exited']

Разделили целевой признак и объясняющие. Столбцы `row_number`, `surname`, `customer_id` исключили, так как они не являются признаками, оказывающими влияние на факт ухода клиента.

In [10]:
features_train, features_valid, target_train, target_valid = train_test_split(
    features, target, test_size=0.4, random_state=12345, stratify=target)

In [11]:
features_valid, features_test, target_valid, target_test = train_test_split(
    features_valid, target_valid, test_size=0.5, random_state=12345, stratify=target_valid)

In [12]:
features_train.shape

(6000, 11)

In [13]:
features_valid.shape

(2000, 11)

In [14]:
features_test.shape

(2000, 11)

Исходные данные разделены на обучающую, валидационную и тестовую выборки в соотношении 3/1/1

In [15]:
scaler_columns = ['credit_score', 'age', 'tenure', 'balance', 'num_of_products', 'estimated_salary']

In [16]:
scaler = StandardScaler()
features_train[scaler_columns] = scaler.fit_transform(features_train[scaler_columns])
features_valid[scaler_columns] = scaler.transform(features_valid[scaler_columns])
features_test[scaler_columns] = scaler.transform(features_test[scaler_columns])

/tmp/ipykernel_129/3326936502.py:2: SettingWithCopyWarning: 

A value is trying to be set on a copy of a slice from a DataFrame.

Try using .loc[row_indexer,col_indexer] = value instead



See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  features_train[scaler_columns] = scaler.fit_transform(features_train[scaler_columns])

/opt/conda/lib/python3.9/site-packages/pandas/core/indexing.py:1738: SettingWithCopyWarning: 

A value is trying to be set on a copy of a slice from a DataFrame.

Try using .loc[row_indexer,col_indexer] = value instead



See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  self._setitem_single_column(loc, value[:, i].tolist(), pi)


Стандартизировали данные (привели к одному масштабу). Бинарные данные не использовали в стандартизации.

## Исследование задачи

In [17]:
best_model_f1=None
best_model_roc_auc=None
best_result_f1=0
best_result_roc_auc=0
for depth in range(1,15):
    tree = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    tree.fit(features_train, target_train)
    predictions_valid=tree.predict(features_valid)
    result_f1 = f1_score(target_valid, predictions_valid)
    result_roc_auc = roc_auc_score(target_valid, tree.predict_proba(features_valid)[:, 1])
    if result_f1 > best_result_f1:
        best_model_f1=tree
        best_result_f1=result_f1
        best_result_roc_auc = result_roc_auc
print('Лучшая модель по метрике f1:', best_model_f1)
print('F1:', best_result_f1)
print('ROC-AUC:', best_result_roc_auc)

Лучшая модель по метрике f1: DecisionTreeClassifier(max_depth=7, random_state=12345)

F1: 0.5924855491329479

ROC-AUC: 0.8247056360232534


Для модели "Дерево решений" наилучший результат на валидационный выборке по метрике F1 составляет 0.592 (лучшее значение параметра max_depth = 7), ROC-AUC = 0.825.

In [18]:
best_model_f1=None
best_model_roc_auc=None
best_result_f1=0
best_result_roc_auc=0
for est in range(1,51, 5):
    for depth in range(1,21):
        forest = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth)
        forest.fit(features_train, target_train)
        predictions_valid = forest.predict(features_valid)
        result_f1 = f1_score(target_valid, predictions_valid)
        result_roc_auc = roc_auc_score(target_valid, forest.predict_proba(features_valid)[:, 1])
        if result_f1 > best_result_f1:
            best_model_f1=forest
            best_result_f1=result_f1
            best_result_roc_auc = result_roc_auc
print('Лучшая модель по метрике f1:', best_model_f1)
print('F1:', best_result_f1)
print('ROC-AUC:', best_result_roc_auc)

Лучшая модель по метрике f1: RandomForestClassifier(max_depth=14, n_estimators=31, random_state=12345)

F1: 0.618978102189781

ROC-AUC: 0.8573912762341118


Для модели "Случайный лес" наилучший результат на валидационный выборке по метрике F1 составляет 0.619 (лучшее значение параметра max_depth = 14, n_estimators = 31), ROC-AUC = 0.857. 

In [19]:
regression = LogisticRegression(random_state=12345, solver='lbfgs')
regression.fit(features_train, target_train)
predictions_valid = regression.predict(features_valid)
result_f1 = f1_score(target_valid, predictions_valid)
result_roc_auc = roc_auc_score(target_valid, regression.predict_proba(features_valid)[:, 1])
print('F1:', result_f1)
print('ROC-AUC:', result_roc_auc)

F1: 0.30458715596330277

ROC-AUC: 0.7876468740762637


Для модели "Логистическая регрессия" наилучший результат на валидационный выборке по метрике F1 составляет 0.305, ROC-AUC = 0.788.

Таким образом, лучшой моделью по показателям F1 и ROC-AUC является "Случайный лес" с параметрами max_depth = 14, n_estimators = 31.

## Борьба с дисбалансом

In [20]:
df['exited'].value_counts(normalize=True)

0    0.7963
1    0.2037
Name: exited, dtype: float64

Как видим, баланс классов нарушен. Количество положительных ответов (клиент ушел) составляет всего 20% от всех данных. Избавимся от дисбаланса классов разными методами и сравним качество моделей.

### Class weight

In [21]:
best_model_f1=None
best_model_roc_auc=None
best_result_f1=0
best_result_roc_auc=0
for depth in range(1,21):
    tree = DecisionTreeClassifier(random_state=12345, max_depth=depth, class_weight='balanced')
    tree.fit(features_train, target_train)
    predictions_valid=tree.predict(features_valid)
    result_f1 = f1_score(target_valid, predictions_valid)
    result_roc_auc = roc_auc_score(target_valid, tree.predict_proba(features_valid)[:, 1])
    if result_f1 > best_result_f1:
        best_model_f1=tree
        best_result_f1=result_f1
        best_result_roc_auc = result_roc_auc
print('Лучшая модель по метрике f1:', best_model_f1)
print('F1:', best_result_f1)
print('ROC-AUC:', best_result_roc_auc)

Лучшая модель по метрике f1: DecisionTreeClassifier(class_weight='balanced', max_depth=6, random_state=12345)

F1: 0.575925925925926

ROC-AUC: 0.8234124051630702


In [22]:
best_model_f1=None
best_model_roc_auc=None
best_result_f1=0
best_result_roc_auc=0
for est in range(1,51, 5):
    for depth in range(1,21):
        forest = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth, class_weight='balanced')
        forest.fit(features_train, target_train)
        predictions_valid = forest.predict(features_valid)
        result_f1 = f1_score(target_valid, predictions_valid)
        result_roc_auc = roc_auc_score(target_valid, forest.predict_proba(features_valid)[:, 1])
        if result_f1 > best_result_f1:
            best_model_f1=forest
            best_result_f1=result_f1
            best_result_roc_auc = result_roc_auc
print('Лучшая модель по метрике f1:', best_model_f1)
print('F1:', best_result_f1)
print('ROC-AUC:', best_result_roc_auc)

Лучшая модель по метрике f1: RandomForestClassifier(class_weight='balanced', max_depth=8, n_estimators=11,

                       random_state=12345)

F1: 0.6457883369330453

ROC-AUC: 0.8658018955069463


In [23]:
regression = LogisticRegression(random_state=12345, solver='lbfgs', class_weight='balanced')
regression.fit(features_train, target_train)
predictions_valid = regression.predict(features_valid)
result_f1 = f1_score(target_valid, predictions_valid)
result_roc_auc = roc_auc_score(target_valid, regression.predict_proba(features_valid)[:, 1])
print('f1:', result_f1)
print('roc_auc:', result_roc_auc)

f1: 0.5302897278314311

roc_auc: 0.7937712459355604


Качество модели "Дерево решений" снизилось, а моделей "Случайный лес" и "Логистическая регрессия" увеличилось.  
Лучшой моделью по показателям F1 и ROC-AUC является "Случайный лес" с параметрами max_depth = 8, n_estimators = 11.

### Upsample

In [24]:
def upsample(features, target, repeat):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
    
    features_upsampled, target_upsampled = shuffle(
        features_upsampled, target_upsampled, random_state=12345)
    
    return features_upsampled, target_upsampled

features_upsampled, target_upsampled = upsample(features_train, target_train, 4)

In [25]:
best_model_f1=None
best_model_roc_auc=None
best_result_f1=0
best_result_roc_auc=0
for depth in range(1,21):
    tree = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    tree.fit(features_upsampled, target_upsampled)
    predictions_valid=tree.predict(features_valid)
    result_f1 = f1_score(target_valid, predictions_valid)
    result_roc_auc = roc_auc_score(target_valid, tree.predict_proba(features_valid)[:, 1])
    if result_f1 > best_result_f1:
        best_model_f1=tree
        best_result_f1=result_f1
        best_result_roc_auc = result_roc_auc
print('Лучшая модель по метрике f1:', best_model_f1)
print('F1:', best_result_f1)
print('ROC-AUC:', best_result_roc_auc)

Лучшая модель по метрике f1: DecisionTreeClassifier(max_depth=6, random_state=12345)

F1: 0.575925925925926

ROC-AUC: 0.8234278007685487


In [26]:
best_model_f1=None
best_model_roc_auc=None
best_result_f1=0
best_result_roc_auc=0
for est in range(1,51, 5):
    for depth in range(1,21):
        forest = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth)
        forest.fit(features_upsampled, target_upsampled)
        predictions_valid = forest.predict(features_valid)
        result_f1 = f1_score(target_valid, predictions_valid)
        result_roc_auc = roc_auc_score(target_valid, forest.predict_proba(features_valid)[:, 1])
        if result_f1 > best_result_f1:
            best_model_f1=forest
            best_result_f1=result_f1
            best_result_roc_auc = result_roc_auc
print('Лучшая модель по метрике f1:', best_model_f1)
print('F1:', best_result_f1)
print('ROC-AUC:', best_result_roc_auc)

Лучшая модель по метрике f1: RandomForestClassifier(max_depth=11, n_estimators=41, random_state=12345)

F1: 0.6469920544835414

ROC-AUC: 0.871680707458863


In [27]:
regression = LogisticRegression(random_state=12345, solver='liblinear')
regression.fit(features_upsampled, target_upsampled)
predictions_valid = regression.predict(features_valid)
result_f1 = f1_score(target_valid, predictions_valid)
result_roc_auc = roc_auc_score(target_valid, regression.predict_proba(features_valid)[:, 1])
print('f1:', result_f1)
print('roc_auc:', result_roc_auc)

f1: 0.5246753246753246

roc_auc: 0.7938220514336388


Качество моделей "Логистическая регрессия" и "Дерево решений" почти не изменилось, а модели "Случайный лес" выросло (в сравнении с использованием гиперпараметра class_weight='balanced').

Модель "Случайный лес" показала наилучший результат (F1 = 0.647) при следующих гиперпараметрах: max_depth=11, n_estimators=41.

### Downsample

In [28]:
def downsample(features, target, fraction):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_downsampled = pd.concat(
        [features_zeros.sample(frac=fraction, random_state=12345)] + [features_ones])
    target_downsampled = pd.concat(
        [target_zeros.sample(frac=fraction, random_state=12345)] + [target_ones])
    
    features_downsampled, target_downsampled = shuffle(
        features_downsampled, target_downsampled, random_state=12345)
    
    return features_downsampled, target_downsampled

features_downsampled, target_downsampled = downsample(features_train, target_train, 0.25)

In [29]:
best_model_f1=None
best_model_roc_auc=None
best_result_f1=0
best_result_roc_auc=0
for depth in range(1,21):
    tree = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    tree.fit(features_downsampled, target_downsampled)
    predictions_valid=tree.predict(features_valid)
    result_f1 = f1_score(target_valid, predictions_valid)
    result_roc_auc = roc_auc_score(target_valid, tree.predict_proba(features_valid)[:, 1])
    if result_f1 > best_result_f1:
        best_model_f1=tree
        best_result_f1=result_f1
        best_result_roc_auc = result_roc_auc
print('Лучшая модель по метрике f1:', best_model_f1)
print('F1:', best_result_f1)
print('ROC-AUC:', best_result_roc_auc)

Лучшая модель по метрике f1: DecisionTreeClassifier(max_depth=6, random_state=12345)

F1: 0.5939278937381405

ROC-AUC: 0.8472324859592077


In [30]:
best_model_f1=None
best_model_roc_auc=None
best_result_f1=0
best_result_roc_auc=0
for est in range(1,51, 5):
    for depth in range(1,21):
        forest = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth)
        forest.fit(features_downsampled, target_downsampled)
        predictions_valid = forest.predict(features_valid)
        result_f1 = f1_score(target_valid, predictions_valid)
        result_roc_auc = roc_auc_score(target_valid, forest.predict_proba(features_valid)[:, 1])
        if result_f1 > best_result_f1:
            best_model_f1=forest
            best_result_f1=result_f1
            best_result_roc_auc = result_roc_auc
print('Лучшая модель по метрике f1:', best_model_f1)
print('F1:', best_result_f1)
print('ROC-AUC:', best_result_roc_auc)

Лучшая модель по метрике f1: RandomForestClassifier(max_depth=6, n_estimators=21, random_state=12345)

F1: 0.6320845341018252

ROC-AUC: 0.8657903488028378


In [31]:
regression = LogisticRegression(random_state=12345, solver='liblinear')
regression.fit(features_downsampled, target_downsampled)
predictions_valid = regression.predict(features_valid)
result_f1 = f1_score(target_valid, predictions_valid)
result_roc_auc = roc_auc_score(target_valid, regression.predict_proba(features_valid)[:, 1])
print('F1:', result_f1)
print('ROC-AUC:', result_roc_auc)

F1: 0.5250431778929188

ROC-AUC: 0.792741279929057


Качество моделей "Дерево решений", "Логистическая регрессия" увеличилось, а качетсво модели "Случайный лес" снизилось (в сравнении с увеличением выборки).

Таким образом, лучшее качество показала модель "Случайный лес" при устранении дисбаланса классов путем увеличении выборки (F1 = 0.647) с параметрами max_depth=11, n_estimators=41.

## Тестирование модели

In [33]:
features_upsampled, target_upsampled = upsample(pd.concat([features_train, features_valid]), 
                                                pd.concat([target_train, target_valid]), 4)

In [34]:
forest = RandomForestClassifier(random_state=12345, n_estimators=41, max_depth=11)
forest.fit(features_upsampled, target_upsampled)
predictions_test = forest.predict(features_test)
result_f1 = f1_score(target_test, predictions_test)
result_roc_auc = roc_auc_score(target_test, forest.predict_proba(features_test)[:, 1])
print('F1:', result_f1)
print('ROC-AUC:', result_roc_auc)

F1: 0.5943502824858756

ROC-AUC: 0.8569170094593824


Таким образом, лучшей моделью для прогноза, уйдёт клиент из банка в ближайшее время или нет, является модель "Случайный лес" с параметрами max_depth=11, n_estimators=41, полученная путем увеличения выборки для устранения дисбаланса классов.